<a href="https://colab.research.google.com/github/zehavitc/NLP_Hw1/blob/master/NLP_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls '/content/drive/My Drive/NLP/HW1'

en.csv	fr.csv	it.csv	out.txt  tl.csv
es.csv	in.csv	nl.csv	pt.csv	 ZehavitTextTest.csv


In [0]:
import numpy as np
from collections import Counter, defaultdict
import csv
import math

g_startTag = "<s>"
g_endTag = "</s>"

def _update_grams(text, n_gram, n_counts): 
    #print(f'n_gram is {n_gram}')
    for i in range(0, len(text) - n_gram + 1):
      #print(f'i is {i}')
      t = text[i:i+n_gram-1]
      #print(t)
      n_char = text[i+n_gram-1]
      #print(n_char)
      n_counts[t][n_char] += 1
    #handle start and end of tweet 
    if n_gram == 1:
      n_counts[''][g_startTag] += 1
      n_counts[''][g_endTag] += 1
    if n_gram == 2:
      n_counts[g_startTag][text[0]] += 1
      n_counts[text[len(text)-1]][g_endTag] += 1 
    if n_gram > 2:
      n_counts[f'{g_startTag}{text[0]}'][text[n_gram -2]] += 1
      n_counts[f'{text[len(text) - n_gram + 1: len(text)]}'][g_endTag] += 1
      
def _output_model_to_file(n_counts, model_file_path):
    with open(model_file_path, "w") as model_file:
      counter_1gram = n_counts[0]['']
      counter_2gram = n_counts[1]
      counter_3gram = n_counts[2]
      
      #output 3-gram 
      print(f"3-grams:", file=model_file)
      trigramKeys = list(counter_3gram.keys())
      for tk in trigramKeys:
        #print(f'tk is: {tk}, len of tk is {len(tk)}')
        trigramValues = list(counter_3gram[tk].keys())
        if tk.startswith(g_startTag):
          firstChar = g_startTag
          secondChar = tk[3]
        elif tk.endswith(g_endTag):
          firstChar = tk[0]
          secondChar = g_endTag
        else:
          firstChar = tk[0]
          secondChar = tk[1]
        charsCount = counter_2gram[firstChar][secondChar]
        #print(f'chars count of {firstChar} {secondChar} is {charsCount}')
        for v in trigramValues:
          #print(f'v is: {v}')
          print(f'<{firstChar}><{secondChar}><{v}>\t{math.log(counter_3gram[tk][v]/charsCount, 2)}', file=model_file)
      print("", file=model_file)    
      
      #output 2-gram
      print(f"2-grams:", file=model_file)
      bigramKeys = list(counter_2gram.keys())
      for bk in bigramKeys:
        bigramValues = list(counter_2gram[bk].keys())
        charsCount = counter_1gram[bk]
        for v in  bigramValues:
          print(f'<{bk}><{v}>\t{math.log(counter_2gram[bk][v]/charsCount, 2)}', file=model_file) 
      print("", file=model_file)     
      
      #output 1-gram
      print(f"1-grams:", file=model_file)
      unigramKeys = list(counter_1gram.keys())
      charsCount = sum(list(counter_1gram.values()))
      for k in unigramKeys:
        print(f'<{k}>\t{math.log(counter_1gram[k]/charsCount, 2)}', file=model_file)
      print("", file=model_file)




# In: corpus_file_path: a csv corpus file path - each line is a <TweetID>,<TweetText> 
# Out: model_file_path: model_file 
def	lm(corpus_file_path,	model_file_path):
  with open(corpus_file_path, 'r') as corpus_file:
    reader = csv.DictReader(corpus_file)
    #unigram, bigram, trigram
    n_counts = [defaultdict(Counter), defaultdict(Counter), defaultdict(Counter)]
    for row in reader:
      tweet_text = row["tweet_text"]   
      print(tweet_text)
      for i in range(1,4):       
        _update_grams(tweet_text, i, n_counts[i-1])  
      
    _output_model_to_file(n_counts, model_file_path)
  

In [0]:
corpus_file_path = '/content/drive/My Drive/NLP/HW1/ZehavitTextTest.csv'
model_file_path = '/content/drive/My Drive/NLP/HW1/out.txt'
lm(corpus_file_path, model_file_path)

RT @HannahMemories1: This was not only catchy but helpful on my school days #11YearsOfHannahMontana https://t.co/Hd9FRilrAZ
RT @Amour_essie: To argue w me is $250 posting subs $150 to fight me $800.   Now due to the fact that u bitches is broke there shouldn't beâ€¦
